In [ ]:
import pandas as pd

from typing import List, Dict
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


In [ ]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[6]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed


In [ ]:
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_1.run import run_objetivo_1
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_2.run import run_objetivo_2
from app.modules.sga.minpub.report_validator.service.objetivos.validators.objetivo_3.run import run_objetivo_3

In [ ]:

# EXTRACT IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.cuadro_averias import (
    extract_averias_table
) 
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.informe_tecnico import (
    extract_tecnico_reports_without_hours_last_dates
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.extract.anexo_indisponibilidad import (
    extract_indisponibilidad_anexos
)


In [ ]:
# TRANSFORM IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.averias import ( 
    preprocess_df_word_averias
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.informe_tecnico import ( 
    preprocess_df_word_informe_tecnico
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.anexos import ( 
    preprocess_df_word_anexos_indisponibilidad
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_335 import ( 
    preprocess_335
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.sga_380 import ( 
    preprocess_380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.cuismp_sharepoint import ( 
    preprocess_df_cid_cuismp_sharepoint
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.transform.corte_excel import ( 
    preprocess_corte_excel
)


In [ ]:
# MERGE IMPORTS
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.excel_sga.excel_sga import ( 
merge_sga_335_corte_excel_sharepoint_cuismp_sga380
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.cuadro_averias.datos import ( 
merge_word_datos_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.cuadro_averias.telefonia import ( 
merge_word_telefonia_averias_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.datos import ( 
merge_word_datos_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.informe_tecnico.telefonia import ( 
merge_word_telefonia_informe_corte_excel
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.anexo_indisponibilidad.datos import ( 
merge_word_datos_anexos_disponibilidad_df_merged_sga
)
from app.modules.sga.minpub.report_validator.service.objetivos.etl.merge.anexo_indisponibilidad.telefonia import ( 
merge_word_telefonia_anexos_disponibilidad_df_merged_sga
)


In [ ]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 1 -MES 17.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_23-04-2025_28-04-2025_20250505_094111.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB (9)_20250505_093457.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_23-04-2025_28-04-2025_20250505_094643.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS.docx"

In [ ]:
def all_objetivos(
    path_corte_excel, 
    path_sgq_dinamico_335, 
    path_sga_dinamico_380,
    path_cid_cuismp_sharepoint,
    word_datos_file_path,
    word_telefonia_file_path

) -> List[Dict]:
    """
    Calls each objective's validation function and combines the results.
    Each objective function returns a DataFrame with the columns:
      - 'numero de incidencia'
      - 'mensaje'
      - 'objetivo'
    
    Returns:
      A list of dictionaries (one per incident that fails at least one validation)
      across all objectives.
    """
    results = []
    
    # EXTRACT INVOQUE
    df_corte_excel = pd.read_excel(path_corte_excel, skipfooter=2, engine="openpyxl")
    df_sga_dinamico_335 = pd.read_excel(path_sgq_dinamico_335) 
    df_sga_dinamico_380 = pd.read_excel(path_sga_dinamico_380)
    df_cid_cuismp_sharepoint = pd.read_excel(path_cid_cuismp_sharepoint)

    df_word_datos_averias =  extract_averias_table(word_datos_file_path)
    df_word_telefonia_averias = extract_averias_table(word_telefonia_file_path)
    df_word_datos_informe_tec =  extract_tecnico_reports_without_hours_last_dates(word_datos_file_path)
    df_word_telefonia_informe_tec = extract_tecnico_reports_without_hours_last_dates(word_telefonia_file_path)

    raw_datos_anexos = extract_indisponibilidad_anexos(word_datos_file_path)
    raw_tel_anexos  = extract_indisponibilidad_anexos(word_telefonia_file_path)
    cols = ['ticket', 'indisponibilidad_header', 'indisponibilidad_periodos',
            'indisponibilidad_footer', 'indisponibilidad_total']
    df_word_datos_anexos_indis = (
        raw_datos_anexos
        if raw_datos_anexos is not None
        else pd.DataFrame(columns=cols)
    )
    df_word_telefonia_anexos_indis = (
        raw_tel_anexos
        if raw_tel_anexos is not None
        else pd.DataFrame(columns=cols)
    )

    # TRANSFORM INVOQUE
    df_word_datos_averias = preprocess_df_word_averias(df_word_datos_averias)
    df_word_telefonia_averias = preprocess_df_word_averias(df_word_telefonia_averias)
    
    df_word_datos_informe_tec =  preprocess_df_word_informe_tecnico(df_word_datos_informe_tec)
    df_word_telefonia_informe_tec = preprocess_df_word_informe_tecnico(df_word_telefonia_informe_tec)

    df_word_datos_anexos_indis    = preprocess_df_word_anexos_indisponibilidad(df_word_datos_anexos_indis)
    df_word_telefonia_anexos_indis = preprocess_df_word_anexos_indisponibilidad(df_word_telefonia_anexos_indis)

    df_sga_dinamico_335 = preprocess_335(df_sga_dinamico_335)
    df_sga_dinamico_380 = preprocess_380(df_sga_dinamico_380)
    df_corte_excel = preprocess_corte_excel(df_corte_excel)
    df_cid_cuismp_sharepoint = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
    

    # MERGE INVOQUE

    #  SGA 335 - 380 - SHAREPOINT - CORTE - BOTH
    df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )
    
    #  SGA 335 - 380 - SHAREPOINT - CORTE - LEFT ONLY
    df_unmatched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel,
        df_sga_dinamico_335,
        df_cid_cuismp_sharepoint,
        df_sga_dinamico_380,
        'left_only'
        )
    
    # AVERIAS - DATOS - EXCEL
    df_matched_word_datos_averias_corte_excel = merge_word_datos_averias_corte_excel(
        df_corte_excel,
        df_word_datos_averias,
        'both'
        )
   
    # AVERIAS - TELEFONIA - EXCEL
    df_matched_word_telefonia_averias_corte_excel = merge_word_telefonia_averias_corte_excel(
        df_word_telefonia_averias,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - DATOS - EXCEL
    df_matched_word_datos_informe_tecnico_corte_excel = merge_word_datos_informe_corte_excel(
        df_word_datos_informe_tec,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - TELEFONIA - EXCEL
    df_matched_word_telefonia_informe_tecnico_corte_excel = merge_word_telefonia_informe_corte_excel(
        df_word_telefonia_informe_tec,
        df_corte_excel,
        'both'
        )
    
    #ANEXOS INDISPONIBILIDAD - DATOS - EXCEL
    df_matched_word_datos_anexo_indisponibilidad_df_merged_sga = merge_word_datos_anexos_disponibilidad_df_merged_sga(
        df_word_datos_anexos_indis,
        df_matched_corte_sga335_Sharepoint_cuismp_sga380,
        'both'
        )
    
    #ANEXOS INDISPONIBILIDAD - TELEFONIA - EXCEL
    df_matched_word_telefonia_anexo_indisponibilidad_df_merged_sga = merge_word_telefonia_anexos_disponibilidad_df_merged_sga(
        df_word_telefonia_anexos_indis,
        df_matched_corte_sga335_Sharepoint_cuismp_sga380,
        'both'
        )
    

    
#  RUN OBJETIVOS 

    obj1_df = run_objetivo_1(
        df_matched_corte_sga335_Sharepoint_cuismp_sga380,
        df_unmatched_corte_sga335_Sharepoint_cuismp_sga380
        )

    obj2_df = run_objetivo_2(
        df_matched_word_datos_averias_corte_excel,
        df_matched_word_telefonia_averias_corte_excel,
        df_matched_word_datos_informe_tecnico_corte_excel,
        df_matched_word_telefonia_informe_tecnico_corte_excel
        )
    
    obj3_df = run_objetivo_3(
        df_matched_word_datos_anexo_indisponibilidad_df_merged_sga,
        df_matched_word_telefonia_anexo_indisponibilidad_df_merged_sga
    )   

    results.extend(obj1_df.to_dict(orient='records'))
    results.extend(obj2_df.to_dict(orient='records'))
    results.extend(obj3_df.to_dict(orient='records'))

    df_all = pd.DataFrame(results)

    df_grouped = (
        df_all
        .groupby('nro_incidencia', as_index=False)
        .agg({'mensaje': lambda msgs: ' | '.join(msgs),
              'objetivo': lambda objs: ' | '.join(objs), 
              'TIPO REPORTE': 'first', 
              })
    )

    #return df_grouped.to_dict(orient='records')
    return df_grouped


result_df = all_objetivos(
    SAVE_DIR_EXTRACT_EXCEL, 
    SAVE_DIR_EXTRACT_SGA_335, 
    DIR_PARADAS_RELOJ,
    CID_CUISMP_PATH,
    DIR_WORD_DATOS,
    DIR_WORD_TELEFONIA

)

result_df




In [ ]:
from datetime import datetime
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent
timestamp = datetime.now().strftime('%Y-%m-%d_%d_%H%M')
SAVE_FINAL_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "final" / f"validacion_total_{timestamp}.xlsx"

result_df.to_excel(SAVE_FINAL_PATH, index=False)
print(f"file saved")